# Feature Groups Notebook

When you onboard a dataset into Hybrid Intelligence, you can group related *numeric* features together, and these will be treated as a feature group within the ESM.

In this example, we will group the input features `CapitalGain` and `CapitalLoss` into a feature group called `Capital`. You can define feature groups when onboarding a dataset with the `feature_groups` parameter.

# Check Environment Variables
Before installing Hybrid Intelligence in the notebook you need to set these Environment Variables externally as described in the User Guide https://docs.umnai.com/set-up-your-environment. 
This section checks that the environment variables have been set correctly and throws an error if not.

In [1]:
import os

umnai_env_vars = {
    'UMNAI_CLIENT_ID',
    'UMNAI_CLIENT_SECRET',
    'PIP_EXTRA_INDEX_URL',
}

if not umnai_env_vars.issubset(os.environ.keys()):
    raise ValueError(
        'UMNAI environment variables not set correctly. They need to be set before using the Umnai library.'
    )

# Install Hybrid Intelligence
Next we install the UMNAI Platform.

In [2]:
%pip install umnai-platform --quiet

Note: you may need to restart the kernel to use updated packages.


# Set Workspace Paths According to Your Environment
Now we will set the workspace path and the experiment path automatically. They will be set to a local path if you are using a local machine environment or to a Databricks path if you are using a Databricks environment.

## Install Databricks SDK

This checks if you are running on Databricks and installs their SDK if you are.

In [3]:
import os
if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    %pip install databricks-sdk --quiet

If you are on Databricks, you can select whether you would like the workspace to be created in the shared area (available to all users in your account) or in your personal user account area. You can ignore this if you are running on a local environment.

In [4]:
# Set to 1 if you want to use shared or 0 to use personal user account area.
USE_SHARED_WORKSPACE = 1 

## Set Paths
Next the workspace and experiment paths are set automatically.

In [5]:
import os

EXP_NAME = 'featuregroups_adult_income'
if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    from databricks.sdk import WorkspaceClient
    w = WorkspaceClient()

    # # For a Databricks Environment
    WS_PATH = '/dbfs/FileStore/workspaces/'+EXP_NAME
    if USE_SHARED_WORKSPACE:
        EXP_PREFIX = f'/Shared/experiments/'
    else:
        USERNAME = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
        EXP_PREFIX = f'/Users/{USERNAME}/experiments/'
    w.workspace.mkdirs(EXP_PREFIX)
    EXP_PATH = EXP_PREFIX + EXP_NAME
else:
    # For a Local Machine Environment
    WS_PATH = 'resources/workspaces/'+EXP_NAME
    EXP_PATH = EXP_NAME

# Import and Prepare Dataset
Import the dataset to a Pandas DataFrame and the clean data in preparation for onboarding into Hybrid Intelligence.

In [6]:
import pandas as pd
import numpy as np

# Import Adult Income Dataset to pandas dataframe: 
# This dataset can be downloaded from https://archive.ics.uci.edu/dataset/2/adult 
column_names = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus", "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]
dataset_df = pd.read_csv('https://raw.githubusercontent.com/umnaibase/umnai-examples/main/data/adult.data', names = column_names)

# Data Preparation:
dataset_df = dataset_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)    # Remove whitespaces
dataset_df["Income"] = np.where((dataset_df["Income"] == '<=50K'), 0, 1)                # Replace Target values with [0,1]
dataset_df.tail(5)

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,1


# Create or Open a Hybrid Intelligence Workspace
Workspaces are used by the Hybrid Intelligence framework to organize your data and models together in one place.

In [7]:
from umnai.workspaces.context import Workspace

# Open a workspace
ws = Workspace.open(
    path=WS_PATH,
    experiment=EXP_PATH
)

ws # Prints workspace details to confirm created/opened

2023-08-30 14:30:55.058389: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 14:30:55.090080: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 14:30:55.091136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 14:30:56.115896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_12633/470028690.py:4: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  ws = Workspace.open(


# Onboard Hybrid Intelligence Dataset

Onboard the Pandas DataFrame into a Hybrid Intelligence dataset. 

You can specify feature groups by passing a dictionary where the key is the name of the group, and the values are the list of features to group together, to the `feature_groups` parameter.

In [8]:
from umnai.data.datasets import Dataset
from umnai.data.enums import PredictionType

dataset = Dataset.from_pandas(
    dataset_df,
    prediction_type=PredictionType.CLASSIFICATION,
    features=list(dataset_df.drop(['Income'], axis=1).columns),    # All columns except 'Income' are features
    targets=['Income'],
    feature_groups={'Capital': ['CapitalGain', 'CapitalLoss']}
)

dataset # Prints dataset details to confirm created/opened

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/30 14:31:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[ObservationSpec] - MLFLOW Run ID: 72134c5aad9a4c25a6509a2d3af2e287:   0%|          | 0/60 [00:00<?, ?it/s]

23/08/30 14:31:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_

2023-08-30 14:32:12.488431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'fnlwgt' with dtype int64 and shape [?,1]
	 [[{{node fnlwgt}}]]
2023-08-30 14:32:12.653325: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'HoursPerWeek' with dtype int64 and shape [?,1]
	 [[{{node HoursPerWeek}}]]
2023-08-30 14:32:12.689240: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'EducationNum' with dtype int64 and shape [?,1]
	 [[{{node EducationNum}}]

INFO:tensorflow:Assets written to: /opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/featuregroups_adult_income/preprocessing/dataset_name=Dataset_21642972/assets


Dataset(id=85dd7776-60e4-4c76-a056-75f6f85ae9cd; name=Dataset_21642972; is_named=False; workspace_id=None)

# Confirm Feature Group Onboarding
When onboarding a dataset, all user defined feature groups are included in the dataset metadata.

In [9]:
dataset.feature_groups

{'Capital': ['CapitalGain', 'CapitalLoss']}

# Statistical Data
The statistical data for a feature group is still shown in terms of the individual input features.

In [10]:
pd.DataFrame(dataset.stats)

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
minimum,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,0.000000
maximum,90.000000,NaN,1.484705e+06,NaN,16.000000,NaN,NaN,NaN,NaN,NaN,99999.000000,4356.000000,99.000000,NaN,1.000000
mean,38.581647,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,0.240810
stddev,13.640433,NaN,1.055500e+05,NaN,2.572720,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,0.427581
null_count,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
unique_count,NaN,9.0,NaN,16.0,NaN,7.0,15.0,6.0,5.0,2.0,NaN,NaN,NaN,42.0,2.000000


# Induce a Hybrid Intelligence Model

You do not need to make any alterations to the standard procedure to induce a model from a dataset that includes feature groups.

Pre-induced models are available in the notebook workspace on Github and may be downloaded and saved locally. Using Pre-induced models will speed up the execution of the notebook.

If `LOAD_PREINDUCED_MODEL` is set to `1` (default), the notebook will look for and load the pre-induced model with `ESM_ID`. Otherwise, if set to `0` or the pre-induced model is not found, a new model will be induced and saved to the workspace.

In [11]:
# Set this variable to '1' to load a pre-induced model, otherwise set to '0' to re-induce a new model from the dataset
LOAD_PREINDUCED_MODEL = 1

# Model ID
ESM_ID='Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3ad96'

#### Load or Induce the Model

In [12]:
from umnai.esm.model import ESM
from umnai.induction.inducer import ModelInducer

# Check if a saved model with the ESM_ID exists. If it exists load it, otherwise induce a new model, save it and print the model and run IDs
if (LOAD_PREINDUCED_MODEL == 1):
    try:    
        esm = ESM.from_workspace(id = ESM_ID)
        print('Pre-induced ESM loaded from workspace: ' + esm.id)
    except OSError:
        print("No model found in workspace.")
        LOAD_PREINDUCED_MODEL = 0

if (LOAD_PREINDUCED_MODEL == 0):
    print("Inducing a new model - this may take some time.")
    # Induce a simple model quickly using fast execution parameters
    model_inducer = ModelInducer(
        max_interactions=3,
        max_interaction_degree=2,
        max_polynomial_degree=2,
        trials=2,
        estimators=2,
        batch_size=512,
        iterations=2,
    )

    # # Induce a more realistic model using default Induction parameters:
    # model_inducer = ModelInducer()

    # Create an ESM using Induction
    esm = model_inducer.induce(dataset)

    # Save the ESM to your workspace
    esm.save_to_workspace()

    # Note ESM ID and MLFLow Run ID
    print("ESM ID: ", esm.id)
    print("MLflow Run ID: ", esm.producer_run_id)


/usr/local/lib/python3.11/site-packages/keras/layers/serialization.py:265: UmnaiRuntimeWarning: Loading an ESM from an earlier Umnai version: 0.10.0


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_

# Inference: Query a Model 
When you query a Hybrid Intelligence model you get predictions together with explanations in real-time.

## Create a Query with Feature Groups

**No alterations are necessary** to create a query for a model with feature groups, the inputs are still applied to the original feature names.

In [13]:
from umnai.explanations.local import Query
import pandas as pd

query = Query({
    'Age': [39],
    'WorkClass': ['State-gov'],
    'fnlwgt': [77516],
    'Education': ['Bachelors'],
    'EducationNum': [13],
    'MaritalStatus': ['Never-married'],
    'Occupation': ['Adm-clerical'],
    'Relationship': ['Not-in-family'],
    'Race': ['White'],
    'Gender': ['Male'],
    'CapitalGain': [2174],
    'CapitalLoss': [0],
    'HoursPerWeek': [40],
    'NativeCountry': ['United-States']
})

### Instantiate a Local Explainer
Create a LocalExplainer to define the ESM you want to query. The local explainer lets you extract query explanations and predictions in real-time.

In [14]:
from umnai.explanations.local import LocalExplainer

# Instantiate a LocalExplainer:
local_explainer = LocalExplainer(esm)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


## Query Result with Feature Groups

When you are viewing a query result, the attributions will be in terms of the **feature group**.

### Submit the Query
Then you should pass the Query object to the local explainer instance. This will return a QueryResult that includes the prediction and the query explanation.

In [15]:
# Query the model:
query_result = local_explainer(query)

#  Display the Query Result together with the explanation
query_result.data

{'query_input': {'Age': array([39]),
  'WorkClass': array(['State-gov'], dtype=object),
  'fnlwgt': array([77516]),
  'Education': array(['Bachelors'], dtype=object),
  'EducationNum': array([13]),
  'MaritalStatus': array(['Never-married'], dtype=object),
  'Occupation': array(['Adm-clerical'], dtype=object),
  'Relationship': array(['Not-in-family'], dtype=object),
  'Race': array(['White'], dtype=object),
  'Gender': array(['Male'], dtype=object),
  'CapitalGain': array([2174]),
  'CapitalLoss': array([0]),
  'HoursPerWeek': array([40]),
  'NativeCountry': array(['United-States'], dtype=object)},
 'scenario_id': None,
 'context_id': None,
 'query_row_hash': array([37176207294173173271185210056503540525], dtype=object),
 'query_created_time': datetime.datetime(2023, 8, 30, 14, 32, 58, tzinfo=<UTC>),
 'model_id': 'Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3ad96',
 'model_intercept': 0.22852412,
 'dataset_id': 'f94c7d798edf46e797f2d524a7c66b78',
 'run_id': 'edd2e6c1cc3142c28def8d6faf

# Explore and Explain a Model with Feature Groups

When you are exploring a model, the feature module and any interaction modules will be in terms of the **feature group**, while module rules will be in terms of the individual features in the group.

## ModelSummaryView
The Model Summary View gives you an overview of the key parameters, inputs and outputs of the model, and of each module within it.

In [16]:
from umnai.views.model_summary import ModelSummaryView

model_summary_view = ModelSummaryView(esm=esm)
model_summary_view.data

{'model_id': 'Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3ad96',
 'model_name': 'esm',
 'model_title': None,
 'model_created': datetime.datetime(2023, 8, 16, 17, 0, 12, tzinfo=<UTC>),
 'model_last_trained': datetime.datetime(2023, 8, 16, 17, 0, 12, tzinfo=<UTC>),
 'model_uvc': '07957c8392c500dc20f4583b1e0df03d36facc63bd9a11095f5f7ed0331bed32',
 'model_intercept': 0.2285241186618805,
 'has_personal_individual_data': False,
 'has_reuse_restrictions': False,
 'model_doi': '',
 'model_copyright': '',
 'n_input_features': 14,
 'n_transformed_features': 108,
 'n_output_targets': 1,
 'features': ['Age',
  'WorkClass',
  'fnlwgt',
  'Education',
  'EducationNum',
  'MaritalStatus',
  'Occupation',
  'Relationship',
  'Race',
  'Gender',
  'CapitalGain',
  'CapitalLoss',
  'HoursPerWeek',
  'NativeCountry'],
 'targets': ['Income'],
 'n_modules': 15,
 'n_partitions': 28,
 'max_interaction_degree': 2,
 'model_interaction_count': 32,
 'max_width': 2,
 'max_depth': 4,
 'n_categorical_features': 8,

## PartialDependencyView
The Partial Dependency View for the Feature Group module shows you the transfer function of the feature group components (input features) to the  module attribution.

In [17]:
from umnai.views.partial_dependency import PartialDependencyView

# Select Feature Group module
selected_module = 'Capital'

# Generate the view
partial_dependency_view = PartialDependencyView(esm=esm, module=selected_module, use_training_data=False)

# Display the results
partial_dependency_view.data

,input_feature.CapitalGain,input_feature.CapitalLoss,attribution.Income,attribution_normalized.Income,module_partition_index,rule_id,condition_expr_friendly,attribution_delta.Income
0,0,0,-0.343493,-0.000003,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,NaN
1,0,51,-0.313052,-0.000003,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.030442
2,0,57,-0.309470,-0.000002,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.003581
3,0,121,-0.271269,-0.000002,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.038202
4,0,147,-0.255749,-0.000002,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.015519
...,...,...,...,...,...,...,...,...
10297,99999,4203,25.317207,0.000203,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.026264
10298,99999,4219,25.326757,0.000203,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.009550
10299,99999,4273,25.358990,0.000204,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.032232
10300,99999,4330,25.393013,0.000204,0,a6d8d68696e8413c80df46cc1123c9b1,FOR ALL,0.034023


# Explain a Query with Feature Groups

When you create a Feature Attribution View or an Interaction Attribution View with a feature group, the attributions will appear in terms of the feature group. Additionally, you will see the input values of the feature group component features in the column called `grouped_features`. The expressions in the Interaction Attribution View will include terms that use the input features, rather than the feature group.

## FeatureAttributionView

In [18]:
from umnai.views.feature_attribution import FeatureAttributionView

# Create the view and display the data
feature_attribution_view = FeatureAttributionView(query_result)
feature_attribution_view.data

,input_feature,feature_attribution,feature_attribution_absolute,feature_attribution_normalized,grouped_features,feature_input
0,MaritalStatus,-0.773565,0.773565,0.288542,None,Never-married
1,EducationNum,-0.416892,0.416892,0.155502,None,13
2,Occupation,-0.307941,0.307941,0.114863,None,Adm-clerical
3,WorkClass,-0.281652,0.281652,0.105057,None,State-gov
4,Relationship,-0.195638,0.195638,0.072974,None,Not-in-family
5,Education,-0.167668,0.167668,0.062541,None,Bachelors
6,Capital,0.159834,0.159834,0.059619,"[CapitalGain, CapitalLoss]","{'CapitalGain': 2174, 'CapitalLoss': 0}"
7,HoursPerWeek,-0.112036,0.112036,0.041790,None,40
8,Gender,-0.089090,0.089090,0.033231,None,Male
9,Age,0.059649,0.059649,0.022249,None,39


## InteractionAttributionView

In [19]:
from umnai.views.interaction_attribution import InteractionAttributionView

# Create the view and display the data
interaction_attribution_view = InteractionAttributionView(query_result)
interaction_attribution_view.data

,module_id,module_index,module_name,module_partition_index,global_partition_index,rule_id,output_target_index,total_attribution,total_attribution_normalized,input_feature_0,grouped_features_0,input_feature_1,grouped_features_1,feature_attribution_0,feature_attribution_1,feature_input_0,feature_input_1,condition_expr_friendly,summarized_then_expr
0,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,7,MaritalStatus,1,16,4dae8b332425476f91038904a8d2cef7,0,-0.879848,0.260491,MaritalStatus,None,None,None,-0.879848,NaN,Never-married,None,"MaritalStatus = ""Never-married""",-0.879348158836365 - 0.000500208989251405*(Ma...
1,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,14,EducationNum x MaritalStatus,0,26,bba2e98f8c6242f7bab40d566141aba3,0,-0.658962,0.195095,EducationNum,None,MaritalStatus,None,-0.765245,0.106283,13,Never-married,"MaritalStatus ≠ ""Married-civ-spouse""",-0.692275285720825 + 0.00297585766139877*Educ...
2,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,2,EducationNum,1,8,46d48965d32d4f998c9e6899adeef530,0,0.348354,0.103135,EducationNum,None,None,None,0.348354,NaN,13,None,EducationNum > 9.5,0.340601056814194 + 0.000323089694690911*Educ...
3,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,9,Occupation,0,19,4348ce8f6ef04c8d9c8e6d3aef6a69fb,0,-0.307941,0.091170,Occupation,None,None,None,-0.307941,NaN,Adm-clerical,None,FOR ALL,-0.513626337051392 + 0.205685168504715*(Occup...
4,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,12,WorkClass,0,23,517cefe01c094aafb1a9522eb392c6be,0,-0.281652,0.083387,WorkClass,None,None,None,-0.281652,NaN,State-gov,None,FOR ALL,-0.143835604190826 - 0.137816846370697*(WorkC...
5,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,13,Education x Relationship,0,24,a17d967848a243ada10615b92c8708a9,0,-0.219979,0.065128,Relationship,None,Education,None,-0.155207,-0.064772,Not-in-family,Bachelors,"Relationship ≠ ""Husband""",-0.824742138385773 + 0.426693677902222*(Relat...
6,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,4,Capital,0,12,a6d8d68696e8413c80df46cc1123c9b1,0,0.159834,0.047321,Capital,"[CapitalGain, CapitalLoss]",None,None,0.159834,NaN,"{'CapitalGain': 2174, 'CapitalLoss': 0}",None,FOR ALL,-0.343493491411209 + 0.000231521589852450*Cap...
7,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,3,HoursPerWeek,1,10,b2b4763f687c4a35bff342cc07340520,0,-0.112036,0.033170,HoursPerWeek,None,None,None,-0.112036,NaN,40,None,(HoursPerWeek > 35.5) and (HoursPerWeek ≤ 42.5),-0.113316848874092 + 2.55601712948459e-5*Hour...
8,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,5,Education,0,13,c5b13a5ad55d470b9496386b59737b2a,0,-0.102896,0.030464,Education,None,None,None,-0.102896,NaN,Bachelors,None,FOR ALL,-0.189609274268150 + 0.0867131277918816*(Educ...
9,Dataset_28325b6e_0381e2a24f004a87a6e29d8968a3a...,6,Gender,0,14,b0932c4cb8894557907e5f888309fdac,0,-0.089090,0.026376,Gender,None,None,None,-0.089090,NaN,Male,None,FOR ALL,-0.340344309806824 + 0.251254230737686*(Gende...
